<a href="https://colab.research.google.com/github/manchann/security_project/blob/main/isolation_forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
dataset = pd.read_csv('/content/drive/MyDrive/BigData/04_hashed.csv')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
dataset_split = dataset

In [3]:
dataset_split_without_Rdate = dataset_split.drop('Rdate',axis=1)

In [4]:
dataset_split = dataset_split_without_Rdate

In [5]:
dataset_split

,src_ip,dst_ip,Proto,src_port,dst_port,Action,src_country,dst_country
0,154.58.159.102,103.177.12.42,6,52897,445,2,None,US
1,154.58.159.20,125.66.92.196,6,60579,445,2,None,DE
2,154.58.159.164,117.121.178.223,6,63831,445,2,None,US
3,154.58.159.165,205.34.95.97,6,55241,445,2,None,US
4,154.58.159.102,93.56.164.131,6,52898,445,2,None,US
...,...,...,...,...,...,...,...,...
5820305,12.150.252.150,102.116.52.100,6,64259,445,2,None,US
5820306,12.150.252.150,167.212.157.124,6,64261,445,2,None,CN
5820307,145.217.2.97,72.192.214.54,6,25646,23,2,ES,KR
5820308,154.58.159.164,185.151.199.60,6,54633,445,2,None,NL


In [6]:
# dataset_train = dataset.iloc[:100]
# dataset_test = dataset.iloc[100:150]

In [7]:
# IPv4 전처리
def transform_ip(ip): 
  groups = ip.split(".") 
  equalize_group_length = "".join( map( lambda group: group.zfill(3), groups )) 
  return equalize_group_length 

from sklearn.preprocessing import LabelEncoder

# Feature 전처리
def preprocess_df(df):
  
  # IPv4 전처리
  df['src_ip'] = df.src_ip.apply(lambda ip : transform_ip(ip))
  df['dst_ip'] = df.dst_ip.apply(lambda ip : transform_ip(ip))

  # country 전처리
  label_encoder = LabelEncoder()
  df['src_country'] = label_encoder.fit_transform(df['src_country'])
  df['dst_country'] = label_encoder.fit_transform(df['dst_country'])

  return df

In [8]:
dataset_ip = preprocess_df(dataset_split)

In [9]:
import numpy as np
arr = np.array(dataset_ip)
dataset_ip = pd.DataFrame(arr,columns=dataset_ip.columns)


In [10]:
from sklearn.preprocessing import LabelEncoder
job_encode = LabelEncoder()
dataset_ip['src_country'] = job_encode.fit_transform(dataset_ip['src_country'])
dataset_ip['dst_country'] = job_encode.fit_transform(dataset_ip['dst_country'])
dataset_ip.tail()

,src_ip,dst_ip,Proto,src_port,dst_port,Action,src_country,dst_country
5820305,012150252150,102116052100,6,64259,445,2,121,224
5820306,012150252150,167212157124,6,64261,445,2,121,46
5820307,145217002097,072192214054,6,25646,23,2,47,116
5820308,154058159164,185151199060,6,54633,445,2,121,159
5820309,154058159020,180130243165,6,53644,445,2,121,47


In [11]:
from sklearn.ensemble import IsolationForest

model=IsolationForest(n_estimators=100, max_samples='auto', n_jobs=-1,
                      max_features=8, contamination='auto')

model.fit(dataset_ip.to_numpy())


IsolationForest(max_features=8, n_jobs=-1)

In [12]:
score = model.decision_function(dataset_ip.to_numpy())
anomaly = model.predict(dataset_ip.to_numpy())


In [13]:
dataset_ip['scores']= score
dataset_ip['anomaly']= anomaly

dataset_ip.to_csv('/content/drive/MyDrive/BigData/results_al.csv')
anomaly_data = dataset_ip.loc[dataset_ip['anomaly']==-1] # 이상값은 -1으로 나타낸다.
anomaly_data
anomaly_data.to_csv('/content/drive/MyDrive/BigData/result.csv')

In [16]:
result = pd.read_csv('/content/drive/MyDrive/BigData/result.csv')

result.head()

,Unnamed: 0,src_ip,dst_ip,Proto,src_port,dst_port,Action,src_country,dst_country,scores,anomaly
0,5,121007158110,72192214056,6,42406,40357,2,138,116,-0.065812,-1
1,8,45111016042,197066131012,6,4729,80,2,85,116,-0.096186,-1
2,31,129020011057,72192214000,6,58259,64344,2,138,116,-0.076878,-1
3,36,45111016049,77034200015,6,45756,443,2,85,12,-0.043242,-1
4,38,101008193165,72192214208,6,50854,43389,2,85,116,-0.073663,-1


In [15]:
dataset_ip.head()

,src_ip,dst_ip,Proto,src_port,dst_port,Action,src_country,dst_country,scores,anomaly
0,154058159102,103177012042,6,52897,445,2,121,224,0.102170,1
1,154058159020,125066092196,6,60579,445,2,121,54,0.079925,1
2,154058159164,117121178223,6,63831,445,2,121,224,0.092024,1
3,154058159165,205034095097,6,55241,445,2,121,224,0.084946,1
4,154058159102,093056164131,6,52898,445,2,121,224,0.103200,1
